In [1]:
import sys
!{sys.executable} -m pip install lxml
from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np
from datetime import datetime
import re

In [2]:
headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'}
res = requests.get("https://www.spaceweatherlive.com/en/solar-activity/top-50-solar-flares",headers = headers)
soup = BeautifulSoup(res.text)
table = soup.find('table', attrs = {'class' : 'table table-striped table-responsive-md'})
df= pd.read_html(str(table))
df = pd.DataFrame(df[0])
df.columns = ['rank', 'x_class', 'date', 'region', 'start_time', 'max_time', 'end_time', 'movie']
df = df.drop(columns = ['movie'])
for index, row in df.iterrows():
    start = str(row['date']) + " "+ str(row['start_time'])
    end = str(row['date']) + " " + str(row['end_time'])
    maxs = str(row['date']) + " " + str(row['max_time'])
    df.loc[index, 'start_time'] = datetime.strptime(start,"%Y/%m/%d %H:%M")
    df.loc[index, 'end_time'] = datetime.strptime(end,"%Y/%m/%d %H:%M")
    df.loc[index, 'max_time'] = datetime.strptime(maxs,"%Y/%m/%d %H:%M")
df=df.drop(columns = ['date'])
print(df)

    rank x_class  region           start_time             max_time  \
0      1    X28+     486  2003-11-04 19:29:00  2003-11-04 19:53:00   
1      2    X20+    9393  2001-04-02 21:32:00  2001-04-02 21:51:00   
2      3  X17.2+     486  2003-10-28 09:51:00  2003-10-28 11:10:00   
3      4    X17+     808  2005-09-07 17:17:00  2005-09-07 17:40:00   
4      5   X14.4    9415  2001-04-15 13:19:00  2001-04-15 13:50:00   
5      6     X10     486  2003-10-29 20:37:00  2003-10-29 20:49:00   
6      7    X9.4    8100  1997-11-06 11:49:00  1997-11-06 11:55:00   
7      8    X9.3    2673  2017-09-06 11:53:00  2017-09-06 12:02:00   
8      9      X9     930  2006-12-05 10:18:00  2006-12-05 10:35:00   
9     10    X8.3     486  2003-11-02 17:03:00  2003-11-02 17:25:00   
10    11    X8.2    2673  2017-09-10 15:35:00  2017-09-10 16:06:00   
11    12    X7.1     720  2005-01-20 06:36:00  2005-01-20 07:01:00   
12    13    X6.9    1263  2011-08-09 07:48:00  2011-08-09 08:05:00   
13    14    X6.5    

In [3]:
headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'}
res = requests.get("https://cdaw.gsfc.nasa.gov/CME_list/radio/waves_type2.html",headers = headers)
soup = BeautifulSoup(res.text)
table = soup.find('pre').prettify()
matches = re.findall(r'\d{4}/\d{2}/\d{2} \d{2}:\d{2}.+PHTX<',table)
table = []
for x in matches:
    date = re.match(r'\d{4}/\d{2}/\d{2} \d{2}:\d{2} \d{2}/\d{2} \d{2}:\d{2} (\d+\s{4}\d+\s{4}\w{6}\s+\d{4,5})*',x)
    rest = re.findall(r'>([^>]*)<',x)
    comp_str = date[0]
    
    for y in rest:
        try:
            comp_str = comp_str + " " +y
        except IndexError:
            pass
    table.append(comp_str.split())
df2 = pd.DataFrame(columns = ['start_date','start_time','end_date','end_time','start_fz','end_fz','flare_loc','flare_region','flare_class','cme_date','cme_time','CPA','cme_width','cme_speed','plot'], data = table)
df2 = df2.replace('-----',value = np.nan)
df2 = df2.replace('--:--',value = np.nan)
df2 = df2.replace('--/--',value = np.nan)
df2 = df2.replace('----',value = np.nan)
df2 = df2.replace('????',value = np.nan)
df2['is_halo'] = False
df2['has_lower_bound'] = False
halos = df2['CPA'].str.find('Halo')
bounds = df2['cme_width'].str.find('&gt;')
for x in range(0,len(halos)):
    if halos[x] == 0:
        df2.loc[[x],['is_halo']] = True
    if bounds[x] == 0:
        df2.loc[[x],['has_lower_bound']] = True
        
df2 = df2.replace('&gt;\d+',regex = True, value = np.nan)
df2 = df2.replace('Halo',value = np.nan)  
df2 = df2.replace('None', value = np.nan)

for index, row in df2.iterrows():
    year = str(row['start_date'])[:4] +"/"
    start = str(row['start_date']) + " "+ str(row['start_time'])
    end = year + str(row['end_date']) + " " + str(row['end_time'])
    maxs = year + str(row['cme_date']) + " " + str(row['cme_time'])
    
    if re.search('24:00',start) or re.search('nan',start):
        start = start.replace('24:00','00:00')
    if re.search('24:00',end) or re.search('nan',end):
        end = end.replace('24:00','00:00')
    if re.search('24:00',maxs) or re.search('nan',maxs):
        maxs = maxs.replace('24:00','00:00')
    
    if re.search('nan',start) or re.search('None',start):
        df2.loc[index, 'start_datetime'] = np.nan
    else:
        df2.loc[index, 'start_datetime'] = datetime.strptime(start,"%Y/%m/%d %H:%M")
    if re.search('nan',end) or re.search('None',end):
        df2.loc[index, 'end_datetime'] = np.nan
    else:
        df2.loc[index, 'end_datetime'] = datetime.strptime(end,"%Y/%m/%d %H:%M")
    if re.search('nan',maxs) or re.search('None',maxs):
        df2.loc[index, 'cme_datetime'] = np.nan
    else:
        df2.loc[index, 'cme_datetime'] = datetime.strptime(maxs,"%Y/%m/%d %H:%M")
df2 = df2.drop(columns = ['start_date','start_time','end_date','end_time','cme_date','cme_time'])
print(df2)
df2.to_csv('data.csv')

    start_fz end_fz flare_loc flare_region flare_class  CPA cme_width  \
0       8000   4000    S25E16         8026        M1.3   74        79   
1      11000   1000    S28E19         8027        C6.8  NaN       360   
2      12000     80    N21W08         8038        C1.3  NaN       360   
3       5000    500    N05W12         8040        M1.3  263       165   
4       6000   2000    S29E25         8088        C1.4  133       155   
..       ...    ...       ...          ...         ...  ...       ...   
513    14000    210    S10W12        12673        M5.5  NaN       360   
514    16000     70    S08W33        12673        X9.3  NaN       360   
515    16000    150    S09W92          NaN        X8.3  NaN       360   
516    16000  13000    N08E48        12680        C3.0  124        96   
517    16000    900   S08E170          NaN         NaN  NaN       360   

    cme_speed  plot  is_halo  has_lower_bound      start_datetime  \
0         312  PHTX    False            False 1997-04-

Question 2:



In [7]:
top =pd.to_datetime(df['start_time'])
df2['rank'] = -1
df2['start_datetime'] = pd.to_datetime(df2['start_datetime'])
for x in range(0,len(top)):
    print(top[x].date())
    for y in range(0,len(df2['start_datetime'])):
        if df2.loc[[y],['start_datetime']] == int(x.date()):
            df2.loc[[y],['rank']] = x   

2003-11-04


AttributeError: 'DataFrame' object has no attribute 'date'